In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import plotly.express as px
import geopandas as gpd

In [2]:
data = pd.read_csv('checkins.dat', sep = "|")

C:\Users\Lilya\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data = data.replace('                   ', 0.00)
data = data.rename(columns={'   id    ':'id', ' user_id ':'user_id', 
                            ' venue_id ':'venue_id', '     latitude      ':'latitude',
                           '     longitude     ':'longitude', '     created_at      ':'created_at'})

In [4]:
data = data[1:-1]
data = data[data['latitude']!=0.0]

In [5]:
# Ра 396634 строках кластеризация будет работать долго. 
# Сократим сабсет до первых 100 тысяч строк. 
subset = data[:100000]

In [6]:
subset['latitude'] = subset['latitude'].astype('float64')
subset['longitude'] = subset['longitude'].astype('float64')

C:\Users\Lilya\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Lilya\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
# Оставляем только столбцы с долготой и широтой
X = np.array(subset.iloc[:, 3:5])
clustering = MeanShift(bandwidth=0.1).fit(X)

In [8]:
subset_lbl = subset
subset_lbl['label'] = clustering.labels_

C:\Users\Lilya\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
# Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. 
# Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. 
# Центры этих кластеров и являются оптимальными для размещения.
subset_gr = subset_lbl.groupby(subset_lbl.label)[['label']].count()
selected_label = np.array(subset_gr[subset_gr.label > 15].index)
subset_selected_label = subset[subset['label'].isin (selected_label)]

In [10]:
center_cluster = pd.DataFrame(clustering.cluster_centers_, columns=['latitude','longitude'])

In [11]:
subset_selected_label['center_latitude'] = subset_selected_label['label'].map(center_cluster['latitude'])
subset_selected_label['center_longitude'] = subset_selected_label['label'].map(center_cluster['longitude'])

C:\Users\Lilya\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Lilya\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
# Координаты офисов
offices = [[33.751277, -118.188740],
          [25.867736, -80.324116],
          [51.503016, -0.075479],
          [52.378894, 4.885084],
          [39.366487, 117.036146],
          [-33.868457, 151.205134]]
name_offices = ['Los Angeles', 'Miami', 'London', 'Amsterdam', 'Beijing', 'Sydney']

In [13]:
banner_lat = subset_selected_label['center_latitude'].unique()
banner_long = subset_selected_label['center_longitude'].unique()

In [41]:
# 20 баннеров надо разместить близ офисов компании
for ind, _ in enumerate(offices):
    distances = []
    for x,_ in enumerate(banner_lat):
        distances.append(np.sqrt((offices[ind][0] - banner_lat[x])**2+(offices[ind][1] - banner_long[x])**2))
        nearest_banners = pd.DataFrame(distances, columns = ['distance']).sort_values(by='distance')[:20]
        nearest_banners['center_latitude'] = nearest_banners.index.map(pd.DataFrame(banner_lat, columns=['latitude'])['latitude'])
        nearest_banners['center_longitude'] = nearest_banners.index.map(pd.DataFrame(banner_long, columns=['longitude'])['longitude'])
        nearest_banners['office_latitude'] = offices[ind][0]
        nearest_banners['office_longitude'] = offices[ind][1]
        nearest_banners['office_location'] = name_offices[ind]
    if ind == 0:
        office_banners = nearest_banners
    else:
        office_banners = pd.concat([office_banners, nearest_banners])

In [47]:
# Ограничиваем расстояние для того, чтобы не попадали слишком отдаленные
office_banners= office_banners[office_banners.distance <=1]
office_banners= office_banners.sort_values(by='distance')

,distance,center_latitude,center_longitude,office_latitude,office_longitude,office_location,marker_color
139,0.007835,-33.860630,151.204776,-33.868457,151.205134,Sydney,orange
270,0.009353,52.372964,4.892317,52.378894,4.885084,Amsterdam,red
355,0.022674,25.845672,-80.318891,25.867736,-80.324116,Miami,green
118,0.050058,51.502991,-0.125537,51.503016,-0.075479,London,blue
18,0.070848,33.809878,-118.148924,33.751277,-118.188740,Los Angeles,purple
165,0.134109,25.785812,-80.217938,25.867736,-80.324116,Miami,green
140,0.167406,25.705350,-80.283429,25.867736,-80.324116,Miami,green
112,0.188876,26.010098,-80.199991,25.867736,-80.324116,Miami,green
184,0.195779,33.888325,-118.048928,33.751277,-118.188740,Los Angeles,purple
93,0.211811,33.872986,-118.362091,33.751277,-118.188740,Los Angeles,purple


In [43]:
# Наименее удаленный баннер от офиса
answer = np.array([office_banners.iloc[0, 3], office_banners.iloc[0, 4]], dtype = 'float')

In [44]:
np.savetxt('temp.txt', answer, fmt = '%1.6f', newline=' ')

In [45]:
color_office = pd.DataFrame({'office':name_offices, 'marker_color':['purple', 'green', 'blue', 'red', 'pink', 'orange']})
office_banners['marker_color'] = office_banners['office_location'].map(color_office.set_index('office')['marker_color'])

In [46]:
import folium
m = folium.Map()
    
for index, row in data_vis.iterrows():
    folium.Circle([row['center_latitude'], row['center_longitude']],
                  radius=15, color=row['marker_color'],
                 popup = row['office_location']).add_to(m)
    
for index, row in data_vis.iterrows():
    folium.Marker([row['office_latitude'], row['office_longitude']],
                  icon=folium.Icon(color=row['marker_color'], icon="info-sign"),
                 popup = row['office_location']).add_to(m)
    
m